In [1]:
!pip -q install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple

# 验证
import paddle
paddle.utils.run_check()
print("Paddle version:", paddle.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.4 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
Paddle version: 3.2.0


/usr/local/lib/python3.12/dist-packages/paddle/pir/math_op_patch.py:219: UserWarning: Value do not have 'place' interface for pir graph mode, try not to use it. None will be returned.
  warnings.warn(


In [8]:
import math
from typing import Literal, Optional

import paddle
import paddle.nn as nn
import paddle.nn.functional as F

# ---------- 通用初始化 ------------------------------------------------------
def init_rsqrt_uniform_(w: paddle.Tensor) -> paddle.Tensor:
    bound = 1.0 / math.sqrt(w.shape[-1])
    noise = paddle.uniform(w.shape, min=-bound, max=bound, dtype=w.dtype)
    w.set_value(noise)
    return w

def init_random_signs_(w: paddle.Tensor) -> paddle.Tensor:
    # 0/1 伯努利 -> *2 -1  => {-1, +1}
    with paddle.no_grad():
        p = paddle.full(w.shape, 0.5, dtype='float32')
        s = paddle.bernoulli(p) * 2.0 - 1.0
        s = paddle.cast(s, w.dtype)
        w.set_value(s)
    return w

# ---------- 基础层 ----------------------------------------------------------
class NLinear(nn.Layer):
    """PackedEnsemble: K 份 Linear 打包 → 输入 (B,K,D), 权重布局 (K, I, O)"""
    def __init__(self, k: int, in_f: int, out_f: int, bias: bool = True):
        super().__init__()
        self.k = k
        self.in_f = in_f
        self.out_f = out_f
        # 按 Paddle 线性层布局 [I, O]
        self.weight = self.create_parameter(shape=[k, in_f, out_f])
        self.bias_e = self.create_parameter(shape=[k, out_f]) if bias else None
        self.reset_parameters()

    def reset_parameters(self):
        init_rsqrt_uniform_(self.weight)
        if self.bias_e is not None:
            init_rsqrt_uniform_(self.bias_e)

    def forward(self, x):                 # x: (B,K,D=I)
        # 转成 (K,B,D) 与 batched matmul 对齐
        xk = paddle.transpose(x, [1, 0, 2])           # (K,B,I)
        # (K,B,I) @ (K,I,O) = (K,B,O)
        yk = paddle.bmm(xk, self.weight)              # (K,B,O)
        y = paddle.transpose(yk, [1, 0, 2])           # (B,K,O)
        if self.bias_e is not None:
            y = y + self.bias_e                       # 广播到 (B,K,O)
        return y

class ScaleEnsemble(nn.Layer):
    """Mini-Ensemble：每层一个 rank-1 缩放向量"""
    def __init__(self, k: int, d: int, init='ones'):
        super().__init__()
        self.k = k
        self.d = d
        self.init = init
        self.weight = self.create_parameter(shape=[k, d])
        self.reset_parameters()

    def reset_parameters(self):
        if self.init == 'ones':
            self.weight.set_value(paddle.ones_like(self.weight))
        else:
            init_random_signs_(self.weight)

    def forward(self, x):                 # (B,K,D)
        return x * self.weight            # 广播到 (B,K,D)

class LinearBE(nn.Layer):
    """
    BatchEnsemble Linear（Paddle 布局）：
        权重 W: [I, O]；前向 y_e = ((x * r_e) @ W) * s_e + b_e
    输入:  x (B,K,I)
    输出:  y (B,K,O)
    """
    def __init__(self, in_f: int, out_f: int, k: int,
                 scale_init='ones', bias: bool = True):
        super().__init__()
        self.k = k
        self.in_f = in_f
        self.out_f = out_f
        # 显式属性名，避免冲突；按 Paddle 线性层布局 [I, O]
        self.weight = self.create_parameter(shape=[in_f, out_f])
        self.r = self.create_parameter(shape=[k, in_f])
        self.s = self.create_parameter(shape=[k, out_f])
        self.use_bias = bias
        self.bias_e = self.create_parameter(shape=[k, out_f]) if bias else None
        self.scale_init = scale_init
        self.reset_parameters()

    def reset_parameters(self):
        init_rsqrt_uniform_(self.weight)
        if self.scale_init == 'ones':
            self.r.set_value(paddle.ones_like(self.r))
            self.s.set_value(paddle.ones_like(self.s))
        else:
            init_random_signs_(self.r)
            init_random_signs_(self.s)
        if self.use_bias:
            init_rsqrt_uniform_(self.bias_e)

    def forward(self, x):                 # (B,K,I)
        xr = x * self.r                                  # (B,K,I)
        # (B,K,I) @ (I,O) = (B,K,O)
        y = paddle.matmul(xr, self.weight)               # (B,K,O)
        y = y * self.s                                   # (B,K,O)
        if self.use_bias:
            y = y + self.bias_e
        return y

# ---------- Backbone MLP -----------------------------------------------------
class MLPBlock(nn.Layer):
    def __init__(self, d_in, d_hid, dropout, act='ReLU'):
        super().__init__()
        Act = getattr(nn, act)
        self.net = nn.Sequential(
            nn.Linear(d_in, d_hid),   # Paddle: weight [d_in, d_hid]
            Act(),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # 允许 (B,K,D) 或 (B,D)；Linear 会在最后一维上工作
        return self.net(x)

class BackboneMLP(nn.Layer):
    def __init__(self, n_blocks: int, d_in: int, d_hidden: int, dropout: float):
        super().__init__()
        blocks = []
        for i in range(n_blocks):
            blocks.append(
                MLPBlock(d_in if i == 0 else d_hidden, d_hidden, dropout)
            )
        self.blocks = nn.LayerList(blocks)

    def forward(self, x):
        for blk in self.blocks:
            x = blk(x)
        return x

# ---------- 工具：递归替换 Linear 为 BE / Packed ---------------------------
def _get_parent_by_path(root: nn.Layer, path_list):
    """根据命名路径拿到父层（最后一个名是子层名）"""
    cur = root
    for p in path_list:
        if hasattr(cur, p):
            cur = getattr(cur, p)
        else:
            sub_layers = getattr(cur, "_sub_layers", None)
            if sub_layers is None or p not in sub_layers:
                raise AttributeError(f"Cannot locate sublayer '{p}' under '{type(cur).__name__}'")
            cur = sub_layers[p]
    return cur

def _replace_linear(module: nn.Layer, k: int, mode: Literal['be', 'packed']):
    """
    遍历 module 的子层，把 nn.Linear 替换为 LinearBE 或 NLinear
    注意：Paddle Linear 的 weight 形状为 [in_features, out_features]
    """
    to_replace = []

    for full_name, layer in module.named_sublayers(include_self=False):
        if isinstance(layer, nn.Linear):
            parts = full_name.split('.')
            parent_path, child_name = parts[:-1], parts[-1]
            parent = _get_parent_by_path(module, parent_path) if parent_path else module

            in_f  = layer.weight.shape[0]  # I
            out_f = layer.weight.shape[1]  # O

            if mode == 'be':
                new_layer = LinearBE(in_f, out_f, k)
                with paddle.no_grad():
                    # 拷贝共享主权重（[I,O]）与偏置（[O]）
                    assert list(new_layer.weight.shape) == list(layer.weight.shape), \
                        f"weight shape mismatch: {new_layer.weight.shape} vs {layer.weight.shape}"
                    new_layer.weight.set_value(layer.weight.clone())
                    if layer.bias is not None and new_layer.bias_e is not None:
                        b = layer.bias.reshape([1, -1]).tile([k, 1])  # (K, O)
                        assert list(new_layer.bias_e.shape) == list(b.shape), \
                            f"bias shape mismatch: {new_layer.bias_e.shape} vs {b.shape}"
                        new_layer.bias_e.set_value(b)
            else:  # 'packed'
                new_layer = NLinear(k, in_f, out_f, bias=layer.bias is not None)
                with paddle.no_grad():
                    # 每个 pack 共享同一权重初值: 原 (I,O) -> (K,I,O)
                    w = layer.weight.unsqueeze(0).tile([k, 1, 1])  # (K,I,O)
                    assert list(new_layer.weight.shape) == list(w.shape), \
                        f"packed weight shape mismatch: {new_layer.weight.shape} vs {w.shape}"
                    new_layer.weight.set_value(w)
                    if layer.bias is not None and new_layer.bias_e is not None:
                        b = layer.bias.unsqueeze(0).tile([k, 1])    # (K,O)
                        assert list(new_layer.bias_e.shape) == list(b.shape), \
                            f"packed bias shape mismatch: {new_layer.bias_e.shape} vs {b.shape}"
                        new_layer.bias_e.set_value(b)

            to_replace.append((parent, child_name, new_layer))

    # 正式替换
    for parent, child_name, new_layer in to_replace:
        if hasattr(parent, child_name):
            setattr(parent, child_name, new_layer)
        else:
            sub_layers = getattr(parent, "_sub_layers", None)
            if sub_layers is None or child_name not in sub_layers:
                raise AttributeError(f"Cannot set sublayer '{child_name}' under '{type(parent).__name__}'")
            parent._sub_layers[child_name] = new_layer

# ---------- TabM 特征提取器 --------------------------------------------------
class TabMFeatureExtractor(nn.Layer):
    """
    arch_type: 'plain' | 'tabm' | 'tabm-mini' | 'tabm-packed'
    返回：
        - reduce=True  → (B,H)
        - reduce=False → (B,K,H)
    """
    def __init__(self,
                 num_features: int,
                 arch_type: Literal['plain', 'tabm', 'tabm-mini', 'tabm-packed']='tabm',
                 k: int = 32,
                 backbone_cfg: Optional[dict] = None,
                 reduce: bool = True):
        super().__init__()
        if arch_type == 'plain':
            k = 1
        self.k = k
        self.reduce = reduce
        cfg = backbone_cfg or dict(n_blocks=3, d_hidden=512, dropout=0.1)
        self.backbone = BackboneMLP(**cfg, d_in=num_features)

        # --- 插入 Ensemble 逻辑 ---
        if arch_type == 'tabm':
            _replace_linear(self.backbone, k, mode='be')
            self.min_adapter = None
        elif arch_type == 'tabm-mini':
            self.min_adapter = ScaleEnsemble(k, num_features, init='random-signs')
        elif arch_type == 'tabm-packed':
            _replace_linear(self.backbone, k, mode='packed')
            self.min_adapter = None
        else:  # plain
            self.min_adapter = None

    def forward(self, x_num: paddle.Tensor):
        """
        x_num : (B, num_features) – 已完成数值化/标准化
        """
        if self.k > 1:
            x = x_num.unsqueeze(1).tile([1, self.k, 1])  # (B,K,D)
        else:
            x = x_num.unsqueeze(1)                        # (B,1,D)

        if self.min_adapter is not None:
            x = self.min_adapter(x)                       # (B,K,D)

        features = self.backbone(x)                       # (B,K,H)
        if self.reduce:
            return features.mean(axis=1)                  # (B,H)
        return features                                   # (B,K,H)

# ---------------- Quick check ----------------
if __name__ == '__main__':
    paddle.seed(123)
    B, D = 8, 30
    x = paddle.randn([B, D])
    # 1) 标准 TabM（BatchEnsemble 替换）
    fe1 = TabMFeatureExtractor(D, arch_type='tabm', k=16, reduce=True)
    out1 = fe1(x)
    print('TabM-BE features:', list(out1.shape))   # (B, H)

    # 2) tabm-mini（只做 rank-1 缩放）
    fe2 = TabMFeatureExtractor(D, arch_type='tabm-mini', k=16, reduce=False)
    out2 = fe2(x)
    print('TabM-mini features:', list(out2.shape)) # (B, K, H)

    # 3) tabm-packed（Packed NLinear）
    fe3 = TabMFeatureExtractor(D, arch_type='tabm-packed', k=8, reduce=True)
    out3 = fe3(x)
    print('TabM-packed features:', list(out3.shape)) # (B, H)

    # 4) plain（无集成基线）
    fe4 = TabMFeatureExtractor(D, arch_type='plain', k=1, reduce=True)
    out4 = fe4(x)
    print('Plain features:', list(out4.shape))     # (B, H)


TabM-BE features: [8, 512]
TabM-mini features: [8, 16, 512]
TabM-packed features: [8, 512]
Plain features: [8, 512]


In [9]:
# -*- coding: utf-8 -*-
import math
from typing import Optional, Literal, Tuple
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader


# ====== 数据集（示例：合成数据）========================================
class ToyMultiLabelDataset(Dataset):
    """
    返回:
      x_num: float32, 形状 (D,)
      y:     float32, 形状 (4,)  —— 多标签 0/1
    """
    def __init__(self, n: int, d: int, seed: int = 123):
        super().__init__()
        rng = np.random.default_rng(seed)
        self.X = rng.normal(size=(n, d)).astype('float32')
        # 随机生成 4 个线性规则 + 噪声，得到多标签
        W = rng.normal(size=(d, 4))
        logits = self.X @ W + rng.normal(scale=0.5, size=(n, 4))
        probs  = 1.0 / (1.0 + np.exp(-logits))
        self.Y = (probs > 0.5).astype('float32')

    def __getitem__(self, idx: int):
        return self.X[idx], self.Y[idx]

    def __len__(self) -> int:
        return len(self.X)

# ====== 模型：特征抽取 + 多标签头 ======================================
class MultiLabelClassifier(nn.Layer):
    def __init__(self, num_features: int, num_labels: int = 4,
                 arch_type: str = 'tabm', k: int = 16,
                 backbone_cfg: Optional[dict] = None):
        super().__init__()
        self.fe = TabMFeatureExtractor(
            num_features=num_features,
            arch_type=arch_type,
            k=k,
            backbone_cfg=backbone_cfg,
            reduce=True
        )
        # 推断隐藏维度（若你的 TabM 有属性可读，直接使用；否则手动传入）
        d_hidden = getattr(self.fe, "d_hidden", (backbone_cfg or dict(d_hidden=512))["d_hidden"])
        self.head = nn.Linear(d_hidden, num_labels)

    def forward(self, x_num: paddle.Tensor) -> paddle.Tensor:
        # x_num: (B, D)
        h = self.fe(x_num)           # (B, H)
        logits = self.head(h)        # (B, 4)
        return logits

# ====== 评价指标：F1、AP 等 ==============================================
def f1_per_class(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-9) -> Tuple[np.ndarray, float, float]:
    """
    y_true: (N, C) 0/1
    y_pred: (N, C) 0/1
    返回: per_class F1, macro-F1, micro-F1
    """
    assert y_true.shape == y_pred.shape
    N, C = y_true.shape
    f1_c = np.zeros(C, dtype=np.float32)

    # per-class
    for c in range(C):
        yt = y_true[:, c]
        yp = y_pred[:, c]
        tp = np.sum((yt == 1) & (yp == 1))
        fp = np.sum((yt == 0) & (yp == 1))
        fn = np.sum((yt == 1) & (yp == 0))
        prec = tp / (tp + fp + eps)
        rec  = tp / (tp + fn + eps)
        f1_c[c] = 2 * prec * rec / (prec + rec + eps)

    macro_f1 = float(np.mean(f1_c))

    # micro
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    prec = tp / (tp + fp + 1e-9)
    rec  = tp / (tp + fn + 1e-9)
    micro_f1 = 2 * prec * rec / (prec + rec + 1e-9)
    return f1_c, macro_f1, float(micro_f1)

def average_precision_micro(y_true: np.ndarray, y_prob: np.ndarray, num_thresholds: int = 101) -> float:
    """
    简易版 micro-AP(AUCPR)：在 0~1 阈值上扫一遍，近似计算 PR 曲线下面积
    """
    thresholds = np.linspace(0.0, 1.0, num_thresholds)
    precision, recall = [], []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(np.float32)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        precision.append(p); recall.append(r)
    # 按 recall 升序进行梯形积分
    order = np.argsort(recall)
    recall = np.array(recall)[order]
    precision = np.array(precision)[order]
    auc_pr = np.trapz(precision, recall)
    return float(auc_pr)

# ====== 训练/验证循环 =====================================================
def train_one_epoch(model, loader, optimizer,
                    pos_weight: Optional[paddle.Tensor] = None,
                    clip_grad_norm: Optional[float] = None,
                    device: str = 'gpu' if paddle.is_compiled_with_cuda() else 'cpu'):
    model.train()
    total_loss = 0.0
    total_batches = 0
    for x, y in loader:
        x = x.astype('float32')
        y = y.astype('float32')
        logits = model(x)
        # BCE with logits（支持 pos_weight）
        if pos_weight is not None:
            loss = F.binary_cross_entropy_with_logits(logits, y, pos_weight=pos_weight)
        else:
            loss = F.binary_cross_entropy_with_logits(logits, y)
        loss.backward()
        if clip_grad_norm is not None:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
        optimizer.step()
        optimizer.clear_grad()
        total_loss += float(loss)
        total_batches += 1
    return total_loss / max(1, total_batches)

@paddle.no_grad()
def evaluate(model, loader, threshold: float = 0.5):
    model.eval()
    ys, ps = [], []
    total_loss, total_batches = 0.0, 0
    for x, y in loader:
        x = x.astype('float32'); y = y.astype('float32')
        logits = model(x)                    # (B,4)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        prob = F.sigmoid(logits).numpy()     # (B,4)
        ys.append(y.numpy())
        ps.append(prob)
        total_loss += float(loss)
        total_batches += 1
    y_true = np.concatenate(ys, axis=0)
    y_prob = np.concatenate(ps, axis=0)
    y_pred = (y_prob >= threshold).astype(np.float32)

    per_f1, macro_f1, micro_f1 = f1_per_class(y_true, y_pred)
    ap_micro = average_precision_micro(y_true, y_prob)
    avg_loss = total_loss / max(1, total_batches)
    metrics = {
        "loss": avg_loss,
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class_f1": per_f1.tolist(),
        "micro_AP": ap_micro
    }
    return metrics

# ====== 主函数：跑通一个最小示例 ===========================================
if __name__ == "__main__":
    paddle.seed(2025)
    # 配置
    D = 30              # 数值特征维度
    C = 4               # 多标签数
    N_train, N_val = 5000, 1000
    batch_size = 128
    epochs = 5
    lr = 3e-4

    # 数据
    train_ds = ToyMultiLabelDataset(N_train, D, seed=42)
    val_ds   = ToyMultiLabelDataset(N_val,   D, seed=233)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, drop_last=False)

    # 类别不平衡（可选）：按训练集估计每个标签的正例比例，构造 pos_weight
    y_train = np.vstack([y for _, y in train_ds])
    pos_ratio = np.clip(y_train.mean(axis=0), 1e-3, 1-1e-3)   # (4,)
    # 经典做法：pos_weight = (N_neg / N_pos) = (1-p)/p
    pos_weight_np = (1.0 - pos_ratio) / pos_ratio
    pos_weight = paddle.to_tensor(pos_weight_np.astype('float32'))  # (4,)

    # 模型
    backbone_cfg = dict(n_blocks=3, d_hidden=512, dropout=0.1)
    model = MultiLabelClassifier(num_features=D, num_labels=C,
                                 arch_type='tabm', k=16,
                                 backbone_cfg=backbone_cfg)

    optimizer = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())

    # 训练
    best_macro_f1, best_state = -1.0, None
    for ep in range(1, epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer,
                                     pos_weight=pos_weight, clip_grad_norm=1.0)
        val_metrics = evaluate(model, val_loader, threshold=0.5)
        print(f"[Epoch {ep:02d}] train_loss={train_loss:.4f} | "
              f"val_loss={val_metrics['loss']:.4f} | "
              f"macro_f1={val_metrics['macro_f1']:.4f} | "
              f"micro_f1={val_metrics['micro_f1']:.4f} | "
              f"per_class_f1={val_metrics['per_class_f1']} | "
              f"micro_AP={val_metrics['micro_AP']:.4f}")
        # 记录最佳
        if val_metrics["macro_f1"] > best_macro_f1:
            best_macro_f1 = val_metrics["macro_f1"]
            best_state = {k: v.clone() for k, v in model.state_dict().items()}

    if best_state is not None:
        model.set_state_dict(best_state)
        print(f"Loaded best state with macro_f1={best_macro_f1:.4f}")


/tmp/ipython-input-1539235308.py:108: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  auc_pr = np.trapz(precision, recall)


[Epoch 01] train_loss=0.4427 | val_loss=1.5854 | macro_f1=0.4728 | micro_f1=0.4734 | per_class_f1=[0.5263158082962036, 0.5373737215995789, 0.4471057951450348, 0.38046795129776] | micro_AP=0.4584
[Epoch 02] train_loss=0.1538 | val_loss=2.9504 | macro_f1=0.4818 | micro_f1=0.4837 | per_class_f1=[0.553903341293335, 0.5406504273414612, 0.44742268323898315, 0.38532111048698425] | micro_AP=0.4769
[Epoch 03] train_loss=0.1057 | val_loss=3.7486 | macro_f1=0.4919 | micro_f1=0.4941 | per_class_f1=[0.5516605377197266, 0.568965494632721, 0.4606299102306366, 0.38624873757362366] | micro_AP=0.4722
[Epoch 04] train_loss=0.0917 | val_loss=4.2100 | macro_f1=0.4762 | micro_f1=0.4774 | per_class_f1=[0.5183752179145813, 0.557729959487915, 0.43551796674728394, 0.3932472765445709] | micro_AP=0.4652
[Epoch 05] train_loss=0.0732 | val_loss=4.7243 | macro_f1=0.4810 | micro_f1=0.4820 | per_class_f1=[0.5422138571739197, 0.5443425178527832, 0.4589178264141083, 0.3785425126552582] | micro_AP=0.4507
Loaded best stat

In [14]:
# -*- coding: utf-8 -*-
import math
from typing import Optional, Literal, Tuple
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader
from paddle.vision.models import resnet18

# ====================== 工具：正弦位置编码 ======================
class SinusoidalPositionalEncoding(nn.Layer):
    def __init__(self, d_model: int, max_len: int = 2048):
        super().__init__()
        pe = np.zeros((max_len, d_model), dtype="float32")
        position = np.arange(0, max_len, dtype="float32")[:, None]
        div_term = np.exp(np.arange(0, d_model, 2, dtype="float32") * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        self.register_buffer("pe", paddle.to_tensor(pe), persistable=False)

    def forward(self, x):  # x: (B, T, D)
        T = x.shape[1]
        return x + self.pe[:T, :]

# ====================== 简化版 TabM（占位，可换成你的实现） ======================
class TabMFeatureExtractor(nn.Layer):
    """占位实现：MLP → (B, H)。可直接替换为你修好的 TabM。"""
    def __init__(self, num_features: int, d_hidden: int = 512, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, d_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_hidden, d_hidden),
            nn.ReLU(),
        )
        self.d_hidden = d_hidden

    def forward(self, x_num: paddle.Tensor):  # (B, 424)
        return self.net(x_num)                # (B, H)

# ====================== ResNet18 特征抽取（逐帧） ======================
class ResNet18FrameEncoder(nn.Layer):
    """将 ResNet18 改为 20 通道输入；输出每帧 512 维特征。"""
    def __init__(self, in_channels: int = 20):
        super().__init__()
        self.backbone = resnet18(pretrained=False)
        # 改首层卷积为 20 通道
        self.backbone.conv1 = nn.Conv2D(in_channels, 64, kernel_size=7, stride=2, padding=3, bias_attr=False)
        # 去掉分类头 fc，保留到 avgpool
        self.avgpool = self.backbone.avgpool  # AdaptiveAvgPool2D(1)
        # 记录下游维度
        self.out_dim = 512

    def forward(self, x):  # x: (B*T, C=20, H=20, W=20)
        m = self.backbone
        x = m.conv1(x); x = m.bn1(x); x = F.relu(x); x = m.maxpool(x)
        x = m.layer1(x); x = m.layer2(x); x = m.layer3(x); x = m.layer4(x)
        x = self.avgpool(x)          # (B*T, 512, 1, 1)
        x = paddle.flatten(x, 1)     # (B*T, 512)
        return x

# ====================== 时序 Transformer 编码器 ======================
class TemporalTransformer(nn.Layer):
    def __init__(self, d_model=512, nhead=8, num_layers=4, dim_feedforward=1024, dropout=0.1, max_len=1024):
        super().__init__()
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                               dim_feedforward=dim_feedforward,
                                               dropout=dropout, activation='relu')
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.pos = SinusoidalPositionalEncoding(d_model, max_len=max_len)

    def forward(self, x):  # x: (B, T, D)
        x = self.pos(x)
        # Paddle 的 Transformer 期望 (T, B, D)
        x = paddle.transpose(x, [1, 0, 2])         # (T,B,D)
        z = self.encoder(x)                        # (T,B,D)
        z = paddle.transpose(z, [1, 0, 2])         # (B,T,D)
        return z

# ====================== 多头注意力（支持 q from A, kv from B） ======================
class MultiHeadCrossAttention(nn.Layer):
    def __init__(self, d_model: int, nhead: int = 8, dropout: float = 0.1):
        super().__init__()
        assert d_model % nhead == 0
        self.d_model = d_model
        self.nhead = nhead
        self.d_head = d_model // nhead
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(d_model)

    def forward(self, q, kv):
        """
        q:  (B, Nq, D)
        kv: (B, Nk, D)
        return: (B, Nq, D)   # 残差 + LN
        """
        B, Nq, D = q.shape
        Nk = kv.shape[1]

        q_lin = self.Wq(q)  # (B,Nq,D)
        k_lin = self.Wk(kv) # (B,Nk,D)
        v_lin = self.Wv(kv) # (B,Nk,D)

        def split_heads(t):  # (B,N,Heads,dh)
            return t.reshape([B, -1, self.nhead, self.d_head]).transpose([0, 2, 1, 3])

        qh = split_heads(q_lin)  # (B,H,Nq,dh)
        kh = split_heads(k_lin)  # (B,H,Nk,dh)
        vh = split_heads(v_lin)  # (B,H,Nk,dh)

        scores = paddle.matmul(qh, kh, transpose_y=True) / math.sqrt(self.d_head)  # (B,H,Nq,Nk)
        attn = F.softmax(scores, axis=-1)
        ctx = paddle.matmul(attn, vh)  # (B,H,Nq,dh)

        ctx = ctx.transpose([0, 2, 1, 3]).reshape([B, Nq, D])  # (B,Nq,D)
        out = self.proj(ctx)
        out = self.drop(out)
        # 残差 + LN
        return self.ln(out + q)

# ====================== 融合头（双向 Cross-Attn） ======================
class BiModalCrossFusion(nn.Layer):
    """
    输入：
      video_seq: (B, T, D) —— Transformer 后的视频序列
      tabm_tok:  (B, D)    —— TabM token
    过程：
      v_token = mean(video_seq)
      v' = CrossAttn(q=v_token[1], kv=tabm_token[1])
      t' = CrossAttn(q=tabm_token[1], kv=video_seq[T])
      fuse = concat([v', t']) → MLP
    """
    def __init__(self, d_model=512, nhead=8, dropout=0.1, fuse_hidden=512):
        super().__init__()
        self.ca_v_from_t = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.ca_t_from_v = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.fuse = nn.Sequential(
            nn.Linear(2 * d_model, fuse_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.out_dim = fuse_hidden

    def forward(self, video_seq, tabm_tok):
        B, T, D = video_seq.shape
        # 池化出视频 token
        v_tok = video_seq.mean(axis=1, keepdim=True)      # (B,1,D)
        t_tok = tabm_tok.unsqueeze(1)                     # (B,1,D)

        v_upd = self.ca_v_from_t(v_tok, t_tok)            # (B,1,D)
        t_upd = self.ca_t_from_v(t_tok, video_seq)        # (B,1,D)

        fused = paddle.concat([v_upd, t_upd], axis=-1)    # (B,1,2D)
        fused = fused.squeeze(1)                          # (B,2D)
        return self.fuse(fused)                           # (B, F)

# ====================== 总模型 ======================
class TwoModalMultiLabelModel(nn.Layer):
    def __init__(self,
                 # 视频模态
                 vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
                 # 结构化模态
                 vec_dim=424,
                 # 维度与结构
                 d_model=512, nhead=8, n_trans_layers=4, trans_ff=1024,
                 tabm_hidden=512, dropout=0.1, num_labels=4):
        super().__init__()
        # A: 逐帧 ResNet18
        self.frame_encoder = ResNet18FrameEncoder(in_channels=vid_channels)  # (B*T,512)
        # A: 时序 Transformer
        self.temporal = TemporalTransformer(d_model=d_model,
                                            nhead=nhead,
                                            num_layers=n_trans_layers,
                                            dim_feedforward=trans_ff,
                                            dropout=dropout,
                                            max_len=vid_frames)
        # B: TabM（或替换为你的 TabM）
        self.tabm = TabMFeatureExtractor(vec_dim, d_hidden=tabm_hidden, dropout=dropout)
        self.tabm_proj = nn.Linear(tabm_hidden, d_model)  # 对齐到 d_model
        # 融合：双向 Cross-Attention
        self.fusion = BiModalCrossFusion(d_model=d_model, nhead=nhead, dropout=dropout, fuse_hidden=d_model)
        # 分类头
        self.head = nn.Linear(self.fusion.out_dim, num_labels)

    def forward(self, x_video, x_vec):
        """
        x_video: (B, T, C=20, H=20, W=20)
        x_vec:   (B, 424)
        """
        B, T, C, H, W = x_video.shape
        # ---- A: 逐帧 ResNet ----
        xvt = x_video.reshape([B * T, C, H, W])          # (B*T, C, H, W)
        f_frame = self.frame_encoder(xvt)                # (B*T, 512)
        f_seq = f_frame.reshape([B, T, -1])              # (B, T, 512)
        # ---- A: 时序 Transformer ----
        z_vid = self.temporal(f_seq)                     # (B, T, 512)
        # ---- B: TabM 特征 ----
        z_tabm = self.tabm(x_vec)                        # (B, H_tabm)
        z_tabm = self.tabm_proj(z_tabm)                  # (B, 512)
        # ---- Cross-Attention 融合 ----
        fused = self.fusion(z_vid, z_tabm)               # (B, 512)
        # ---- 分类 ----
        logits = self.head(fused)                        # (B, 4)
        return logits

# ====================== 指标与训练循环（与前一致） ======================
def f1_per_class(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-9) -> Tuple[np.ndarray, float, float]:
    assert y_true.shape == y_pred.shape
    N, C = y_true.shape
    f1_c = np.zeros(C, dtype=np.float32)
    for c in range(C):
        yt, yp = y_true[:, c], y_pred[:, c]
        tp = np.sum((yt == 1) & (yp == 1))
        fp = np.sum((yt == 0) & (yp == 1))
        fn = np.sum((yt == 1) & (yp == 0))
        prec = tp / (tp + fp + eps)
        rec  = tp / (tp + fn + eps)
        f1_c[c] = 2 * prec * rec / (prec + rec + eps)
    macro_f1 = float(np.mean(f1_c))
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    prec = tp / (tp + fp + 1e-9)
    rec  = tp / (tp + fn + 1e-9)
    micro_f1 = 2 * prec * rec / (prec + rec + 1e-9)
    return f1_c, macro_f1, float(micro_f1)

def average_precision_micro(y_true: np.ndarray, y_prob: np.ndarray, num_thresholds: int = 101) -> float:
    thresholds = np.linspace(0.0, 1.0, num_thresholds)
    precision, recall = [], []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(np.float32)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        precision.append(p); recall.append(r)
    order = np.argsort(recall)
    recall = np.array(recall)[order]
    precision = np.array(precision)[order]
    return float(np.trapz(precision, recall))

def train_one_epoch(model, loader, optimizer,
                    pos_weight: Optional[paddle.Tensor] = None,
                    clip_grad_norm: Optional[float] = None):
    model.train()
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits = model(x_vid.astype('float32'), x_vec.astype('float32'))
        if pos_weight is not None:
            loss = F.binary_cross_entropy_with_logits(logits, y.astype('float32'), pos_weight=pos_weight)
        else:
            loss = F.binary_cross_entropy_with_logits(logits, y.astype('float32'))
        loss.backward()
        if clip_grad_norm is not None:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
        optimizer.step()
        optimizer.clear_grad()
        total_loss += float(loss); total_batches += 1
    return total_loss / max(1, total_batches)

@paddle.no_grad()
def evaluate(model, loader, threshold: float = 0.5):
    model.eval()
    ys, ps = [], []
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits = model(x_vid.astype('float32'), x_vec.astype('float32'))
        loss = F.binary_cross_entropy_with_logits(logits, y.astype('float32'))
        prob = F.sigmoid(logits).numpy()
        ys.append(y.numpy()); ps.append(prob)
        total_loss += float(loss); total_batches += 1
    y_true = np.concatenate(ys, axis=0)
    y_prob = np.concatenate(ps, axis=0)
    y_pred = (y_prob >= threshold).astype(np.float32)
    per_f1, macro_f1, micro_f1 = f1_per_class(y_true, y_pred)
    ap_micro = average_precision_micro(y_true, y_prob)
    return {
        "loss": total_loss / max(1, total_batches),
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class_f1": per_f1.tolist(),
        "micro_AP": ap_micro
    }

# ====================== 合成数据集（可替换为真实数据） ======================
class ToyTwoModalDataset(Dataset):
    """
    返回：
      x_video: (T=365, C=20, H=20, W=20)
      x_vec:   (424,)
      y:       (4,)  0/1
    """
    def __init__(self, n: int, seed: int = 0):
        super().__init__()
        rng = np.random.default_rng(seed)
        self.n = n
        # 按 (n, T, C, H, W)
        self.video = rng.normal(size=(n, 36, 20, 20, 20)).astype('float32')
        self.vec   = rng.normal(size=(n, 424)).astype('float32')

        # 造标签：对视频先在 H/W 上均值，再在 T 上均值 → (n, C=20)
        vid_hw  = self.video.mean(axis=(3, 4))   # (n, T, C)
        vid_avg = vid_hw.mean(axis=1)            # (n, C)

        # 线性映射到 4 个标签
        Wv = rng.normal(size=(20, 4))            # C→4
        Wt = rng.normal(size=(424, 4))           # 424→4
        logits = vid_avg @ Wv + self.vec @ Wt + rng.normal(scale=0.5, size=(n, 4))
        probs  = 1.0 / (1.0 + np.exp(-logits))
        self.y = (probs > 0.5).astype('float32')

    def __getitem__(self, idx: int):
        x_vid = self.video[idx]  # (T,C,H,W)
        x_vec = self.vec[idx]    # (424,)
        y     = self.y[idx]      # (4,)
        return x_vid, x_vec, y

    def __len__(self):
        return self.n

# ====================== 训练入口（可直接运行） ======================
if __name__ == "__main__":
    paddle.seed(2025)
    # 数据
    train_ds = ToyTwoModalDataset(n=64,  seed=42)   # 注意：真实训练建议更大数据与多卡
    val_ds   = ToyTwoModalDataset(n=32,  seed=233)
    # 自定义 collate：让视频变成 (B,T,C,H,W)
    def collate_fn(batch):
        vids, vecs, ys = zip(*batch)
        return (paddle.to_tensor(np.stack(vids, 0)),   # (B,T,C,H,W)
                paddle.to_tensor(np.stack(vecs, 0)),   # (B,424)
                paddle.to_tensor(np.stack(ys, 0)))     # (B,4)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=False, collate_fn=collate_fn)
    val_loader   = DataLoader(val_ds,   batch_size=2, shuffle=False, drop_last=False, collate_fn=collate_fn)

    # 类别不平衡权重（可选）
    y_train = np.stack([y for _, _, y in train_ds], 0)
    pos_ratio = np.clip(y_train.mean(axis=0), 1e-3, 1-1e-3)
    pos_weight = paddle.to_tensor(((1-pos_ratio)/pos_ratio).astype('float32'))  # (4,)

    # 模型
    model = TwoModalMultiLabelModel(
        vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
        vec_dim=424,
        d_model=512, nhead=2, n_trans_layers=2, trans_ff=1024,  # 可调
        tabm_hidden=512, dropout=0.1,
        num_labels=4
    )
    optimizer = paddle.optimizer.Adam(learning_rate=3e-4, parameters=model.parameters())

    # 训练（演示用：小 epoch）
    best_macro_f1, best = -1.0, None
    for ep in range(1, 3+1):
        train_loss = train_one_epoch(model, train_loader, optimizer,
                                     pos_weight=pos_weight, clip_grad_norm=1.0)
        val_metrics = evaluate(model, val_loader, threshold=0.5)
        print(f"[Epoch {ep:02d}] train_loss={train_loss:.4f} | "
              f"val_loss={val_metrics['loss']:.4f} | "
              f"macro_f1={val_metrics['macro_f1']:.4f} | "
              f"micro_f1={val_metrics['micro_f1']:.4f} | "
              f"per_class_f1={val_metrics['per_class_f1']} | "
              f"micro_AP={val_metrics['micro_AP']:.4f}")
        if val_metrics["macro_f1"] > best_macro_f1:
            best_macro_f1 = val_metrics["macro_f1"]
            best = {k: v.clone() for k, v in model.state_dict().items()}

    if best is not None:
        model.set_state_dict(best)
        print(f"Loaded best state with macro_f1={best_macro_f1:.4f}")


/tmp/ipython-input-3157620546.py:248: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(precision, recall))


[Epoch 01] train_loss=1.1304 | val_loss=1.0465 | macro_f1=0.3765 | micro_f1=0.5079 | per_class_f1=[0.5, 0.7450980544090271, 0.260869562625885, 0.0] | micro_AP=0.5393
[Epoch 02] train_loss=0.7661 | val_loss=0.9369 | macro_f1=0.5812 | micro_f1=0.6173 | per_class_f1=[0.5454545617103577, 0.7599999904632568, 0.6382978558540344, 0.380952388048172] | micro_AP=0.4165
[Epoch 03] train_loss=0.4050 | val_loss=1.9395 | macro_f1=0.6107 | micro_f1=0.6303 | per_class_f1=[0.5454545617103577, 0.7234042286872864, 0.6938775777816772, 0.47999998927116394] | micro_AP=0.3836
Loaded best state with macro_f1=0.6107


In [16]:
# -*- coding: utf-8 -*-
import math
from typing import Optional, Tuple
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader
from paddle.vision.models import resnet18

# ====================== 工具：正弦位置编码 ======================
class SinusoidalPositionalEncoding(nn.Layer):
    def __init__(self, d_model: int, max_len: int = 2048):
        super().__init__()
        pe = np.zeros((max_len, d_model), dtype="float32")
        position = np.arange(0, max_len, dtype="float32")[:, None]
        div_term = np.exp(np.arange(0, d_model, 2, dtype="float32") * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        self.register_buffer("pe", paddle.to_tensor(pe), persistable=False)

    def forward(self, x):  # x: (B, T, D)
        T = x.shape[1]
        return x + self.pe[:T, :]

# ====================== 简化版 TabM（占位，可换成你的实现） ======================
class TabMFeatureExtractor(nn.Layer):
    """占位实现：MLP → (B, H)。可直接替换为你修好的 TabM。"""
    def __init__(self, num_features: int, d_hidden: int = 512, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, d_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_hidden, d_hidden),
            nn.ReLU(),
        )
        self.d_hidden = d_hidden

    def forward(self, x_num: paddle.Tensor):  # (B, 424)
        return self.net(x_num)                # (B, H)

# ====================== ResNet18 特征抽取（逐帧） ======================
class ResNet18FrameEncoder(nn.Layer):
    """将 ResNet18 改为 20 通道输入；输出每帧 512 维特征。"""
    def __init__(self, in_channels: int = 20):
        super().__init__()
        self.backbone = resnet18(pretrained=False)
        # 改首层卷积为 20 通道
        self.backbone.conv1 = nn.Conv2D(in_channels, 64, kernel_size=7, stride=2, padding=3, bias_attr=False)
        # 去掉分类头 fc，保留到 avgpool
        self.avgpool = self.backbone.avgpool  # AdaptiveAvgPool2D(1)
        self.out_dim = 512

    def forward(self, x):  # x: (B*T, C=20, H=20, W=20)
        m = self.backbone
        x = m.conv1(x); x = m.bn1(x); x = F.relu(x); x = m.maxpool(x)
        x = m.layer1(x); x = m.layer2(x); x = m.layer3(x); x = m.layer4(x)
        x = self.avgpool(x)          # (B*T, 512, 1, 1)
        x = paddle.flatten(x, 1)     # (B*T, 512)
        return x

# ====================== MoE 基础实现（Top-k，可开关；使用 gather_nd 修复） ======================
class ExpertFFN(nn.Layer):
    def __init__(self, d_model, d_ff, dropout=0.1, act='relu'):
        super().__init__()
        Act = getattr(F, act) if isinstance(act, str) else act
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.drop = nn.Dropout(dropout)
        self.act = Act
    def forward(self, x):
        return self.fc2(self.drop(self.act(self.fc1(x))))

class MoEConfig:
    def __init__(self,
                 n_experts=8,
                 top_k=1,
                 d_ff=2048,
                 dropout=0.1,
                 router_temp=0.5,
                 balance_loss_w=0.005,
                 entropy_reg_w=-0.005,  # 负值→更尖锐
                 diversity_w=1e-3,
                 sticky_w=0.0,
                 sup_router_w=0.0,
                 use_gumbel=True):
        self.n_experts = n_experts
        self.top_k = top_k
        self.d_ff = d_ff
        self.dropout = dropout
        self.router_temp = router_temp
        self.balance_loss_w = balance_loss_w
        self.entropy_reg_w = entropy_reg_w
        self.diversity_w = diversity_w
        self.sticky_w = sticky_w
        self.sup_router_w = sup_router_w
        self.use_gumbel = use_gumbel

class MoE(nn.Layer):
    """forward(x, domain_id=None) → (y, aux_loss)，支持 (B,T,D) 或 (N,D)"""
    def __init__(self, d_model: int, cfg: MoEConfig):
        super().__init__()
        self.cfg = cfg
        self.router = nn.Linear(d_model, cfg.n_experts)
        self.experts = nn.LayerList([ExpertFFN(d_model, cfg.d_ff, cfg.dropout) for _ in range(cfg.n_experts)])
        self.ln = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(cfg.dropout)

    def _router_probs(self, logits):
        if self.cfg.use_gumbel and self.training:
            u = paddle.uniform(logits.shape, min=1e-6, max=1-1e-6, dtype=logits.dtype)
            g = -paddle.log(-paddle.log(u))
            logits = logits + g
        return F.softmax(logits / self.cfg.router_temp, axis=-1)

    def forward(self, x, domain_id=None):
        orig_shape = x.shape
        if len(orig_shape) == 3:
            B, T, D = orig_shape
            X = x.reshape([B*T, D])
        else:
            X = x
        N, D = X.shape

        logits = self.router(X)             # (N,E)
        probs = self._router_probs(logits)  # (N,E)
        topk_val, topk_idx = paddle.topk(probs, k=self.cfg.top_k, axis=-1)  # (N,k)

        # 专家并行输出
        all_out = paddle.stack([e(X) for e in self.experts], axis=1)         # (N,E,D)

        # === 使用 gather_nd 逐样本选择 top-k 专家 ===
        arangeN = paddle.arange(N, dtype='int64')
        picked_list = []
        for i in range(self.cfg.top_k):
            idx_i = topk_idx[:, i].astype('int64')                   # (N,)
            idx_nd = paddle.stack([arangeN, idx_i], axis=1)          # (N,2) [sample, expert]
            picked_i = paddle.gather_nd(all_out, idx_nd)             # (N,D)
            picked_list.append(picked_i)
        picked = paddle.stack(picked_list, axis=1)                   # (N,k,D)

        # 归一化权重并加权
        w = topk_val / (paddle.sum(topk_val, axis=-1, keepdim=True) + 1e-9)  # (N,k)
        Y = paddle.sum(picked * w.unsqueeze(-1), axis=1)                      # (N,D)

        Y = self.drop(Y)
        Y = self.ln(Y + X)

        # aux loss
        aux = 0.0
        if self.cfg.balance_loss_w > 0:
            mean_prob = probs.mean(axis=0)
            target = paddle.full_like(mean_prob, 1.0 / self.cfg.n_experts)
            aux = aux + self.cfg.balance_loss_w * F.mse_loss(mean_prob, target)
        if self.cfg.entropy_reg_w != 0.0:
            ent = -paddle.sum(probs * (paddle.log(probs + 1e-9)), axis=1).mean()
            aux = aux + self.cfg.entropy_reg_w * ent
        if (domain_id is not None) and (self.cfg.sup_router_w > 0):
            dom = domain_id.reshape([-1])[:N] % self.cfg.n_experts
            aux = aux + self.cfg.sup_router_w * F.cross_entropy(logits, dom)
        if self.cfg.diversity_w > 0 and self.cfg.n_experts > 1:
            # 用 top-1 硬选择近似每个专家接收的样本
            chosen = F.one_hot(topk_idx[:, 0], num_classes=self.cfg.n_experts).astype('float32')  # (N,E)
            denom = chosen.sum(axis=0).clip(min=1.0).unsqueeze(-1)
            means = (all_out * chosen.unsqueeze(-1)).sum(axis=0) / denom                           # (E,D)
            sims = []
            for i in range(self.cfg.n_experts):
                for j in range(i+1, self.cfg.n_experts):
                    si = F.normalize(means[i:i+1], axis=-1)
                    sj = F.normalize(means[j:j+1], axis=-1)
                    sims.append((si*sj).sum())
            if sims:
                aux = aux + self.cfg.diversity_w * paddle.stack(sims).mean()

        if len(orig_shape) == 3:
            Y = Y.reshape([B, T, D])
        return Y, aux

class MoEHead(nn.Layer):
    """单 token MoE 头，用于 fused/tabm 投影后的 (B, D)"""
    def __init__(self, d_model=512, cfg: MoEConfig = None):
        super().__init__()
        self.moe = MoE(d_model, cfg or MoEConfig())
    def forward(self, tok, domain_id=None):
        y, aux = self.moe(tok.unsqueeze(1), domain_id=domain_id)  # (B,1,D)
        return y.squeeze(1), aux

# ====================== 自定义 Transformer Encoder（FFN 可替换为 MoE） ======================
class TransformerEncoderLayerMoE(nn.Layer):
    def __init__(self, d_model=512, nhead=8, d_ff=1024, dropout=0.1,
                 use_moe: bool = True, moe_cfg: MoEConfig = None):
        super().__init__()
        self.use_moe = use_moe
        self.self_attn = nn.MultiHeadAttention(embed_dim=d_model, num_heads=nhead, dropout=dropout)
        self.ln1 = nn.LayerNorm(d_model)
        self.do1 = nn.Dropout(dropout)
        if use_moe:
            self.moe = MoE(d_model, moe_cfg or MoEConfig(d_ff=d_ff, dropout=dropout))
        else:
            self.ffn = nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_ff),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(d_ff, d_model),
            )
            self.do2 = nn.Dropout(dropout)

    def forward(self, x, domain_id=None):  # x: (B,T,D)
        # Self-Attention (pre-norm) —— Paddle MHA 期望 (T,B,D)
        h = self.ln1(x)
        h = paddle.transpose(h, [1, 0, 2])          # (T,B,D)
        sa = self.self_attn(h, h, h)                # (T,B,D)
        sa = paddle.transpose(sa, [1, 0, 2])        # (B,T,D)
        x = x + self.do1(sa)
        aux = 0.0
        if self.use_moe:
            x, aux = self.moe(x, domain_id=domain_id)  # 残差+LN 在 MoE 内部
        else:
            x = x + self.do2(self.ffn(x))              # 残差在这里
        return x, aux

class TemporalTransformerFlexible(nn.Layer):
    def __init__(self, d_model=512, nhead=8, num_layers=2, d_ff=1024, dropout=0.1,
                 max_len=1024, use_moe: bool = True, moe_cfg: MoEConfig = None):
        super().__init__()
        self.pos = SinusoidalPositionalEncoding(d_model, max_len=max_len)
        self.layers = nn.LayerList([
            TransformerEncoderLayerMoE(d_model, nhead, d_ff, dropout,
                                       use_moe=use_moe, moe_cfg=moe_cfg)
            for _ in range(num_layers)
        ])
    def forward(self, x, domain_id=None):  # x: (B,T,D)
        x = self.pos(x)
        aux_total = 0.0
        for layer in self.layers:
            x, aux = layer(x, domain_id=domain_id)
            aux_total = aux_total + aux
        return x, aux_total

# ====================== 多头注意力（支持 q from A, kv from B） ======================
class MultiHeadCrossAttention(nn.Layer):
    def __init__(self, d_model: int, nhead: int = 8, dropout: float = 0.1):
        super().__init__()
        assert d_model % nhead == 0
        self.d_model = d_model
        self.nhead = nhead
        self.d_head = d_model // nhead
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(d_model)

    def forward(self, q, kv):
        B, Nq, D = q.shape
        Nk = kv.shape[1]
        q_lin = self.Wq(q); k_lin = self.Wk(kv); v_lin = self.Wv(kv)
        def split_heads(t):
            return t.reshape([B, -1, self.nhead, self.d_head]).transpose([0, 2, 1, 3])
        qh = split_heads(q_lin); kh = split_heads(k_lin); vh = split_heads(v_lin)
        scores = paddle.matmul(qh, kh, transpose_y=True) / math.sqrt(self.d_head)
        attn = F.softmax(scores, axis=-1)
        ctx = paddle.matmul(attn, vh)
        ctx = ctx.transpose([0, 2, 1, 3]).reshape([B, Nq, D])
        out = self.proj(ctx)
        out = self.drop(out)
        return self.ln(out + q)

# ====================== 融合头（双向 Cross-Attn） ======================
class BiModalCrossFusion(nn.Layer):
    """
    输入：
      video_seq: (B, T, D) —— Transformer 后的视频序列
      tabm_tok:  (B, D)    —— TabM token
    """
    def __init__(self, d_model=512, nhead=8, dropout=0.1, fuse_hidden=512):
        super().__init__()
        self.ca_v_from_t = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.ca_t_from_v = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.fuse = nn.Sequential(
            nn.Linear(2 * d_model, fuse_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.out_dim = fuse_hidden

    def forward(self, video_seq, tabm_tok):
        B, T, D = video_seq.shape
        # 池化视频时间维得到 token
        v_tok = video_seq.mean(axis=1, keepdim=True)      # (B,1,D)
        t_tok = tabm_tok.unsqueeze(1)                     # (B,1,D)
        v_upd = self.ca_v_from_t(v_tok, t_tok)            # (B,1,D)
        t_upd = self.ca_t_from_v(t_tok, video_seq)        # (B,1,D)
        fused = paddle.concat([v_upd, t_upd], axis=-1)    # (B,1,2D)
        fused = fused.squeeze(1)                          # (B,2D)
        return self.fuse(fused)                           # (B, F)

# ====================== 总模型（带三个 MoE 开关） ======================
class TwoModalMultiLabelModel(nn.Layer):
    def __init__(self,
                 # 视频模态
                 vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
                 # 结构化模态
                 vec_dim=424,
                 # 维度与结构
                 d_model=512, nhead=2, n_trans_layers=2, trans_ff=1024,
                 tabm_hidden=512, dropout=0.1, num_labels=4,
                 # ===== MoE 开关 =====
                 moe_temporal: bool = True,      # 时序 Transformer 的 FFN 位置
                 moe_fused: bool = False,        # 融合 token 上的小型 MoE 头
                 moe_tabm: bool = False,         # TabM 投影后
                 # ===== MoE 超参（可传入自定义） =====
                 moe_cfg_temporal: MoEConfig = None,
                 moe_cfg_fused: MoEConfig = None,
                 moe_cfg_tabm: MoEConfig = None):
        super().__init__()
        # A: 逐帧 ResNet18
        self.frame_encoder = ResNet18FrameEncoder(in_channels=vid_channels)
        # A: 时序 Transformer（可开/关 MoE）
        self.temporal = TemporalTransformerFlexible(
            d_model=d_model, nhead=nhead, num_layers=n_trans_layers,
            d_ff=trans_ff, dropout=dropout, max_len=vid_frames,
            use_moe=moe_temporal,
            moe_cfg=moe_cfg_temporal or MoEConfig(
                n_experts=8, top_k=1, d_ff=max(trans_ff, 2048), router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            )
        )
        # B: TabM（或你的 TabM）
        self.tabm = TabMFeatureExtractor(vec_dim, d_hidden=tabm_hidden, dropout=dropout)
        self.tabm_proj = nn.Linear(tabm_hidden, d_model)

        # 可选：TabM 分支 MoE 头
        self.moe_tabm = moe_tabm
        if moe_tabm:
            self.tabm_moe = MoEHead(d_model=d_model, cfg=moe_cfg_tabm or MoEConfig(
                n_experts=6, top_k=1, d_ff=1024, router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            ))

        # 融合：双向 Cross-Attention
        self.fusion = BiModalCrossFusion(d_model=d_model, nhead=nhead, dropout=dropout, fuse_hidden=d_model)

        # 可选：融合 token MoE 头
        self.moe_fused = moe_fused
        if moe_fused:
            self.fused_moe = MoEHead(d_model=d_model, cfg=moe_cfg_fused or MoEConfig(
                n_experts=6, top_k=1, d_ff=1024, router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            ))

        # 分类头
        self.head = nn.Linear(self.fusion.out_dim, num_labels)

    def forward(self, x_video, x_vec, domain_id=None):
        """
        x_video: (B, T, C=20, H=20, W=20)
        x_vec:   (B, 424)
        domain_id: (B,) 或 None —— 若有域/季节/站点标签，可传入以做监督路由（可选）
        """
        B, T, C, H, W = x_video.shape
        # ---- A: 逐帧 ResNet ----
        xvt = x_video.reshape([B * T, C, H, W])          # (B*T, C, H, W)
        f_frame = self.frame_encoder(xvt)                # (B*T, 512)
        f_seq = f_frame.reshape([B, T, -1])              # (B, T, 512)
        # ---- A: 时序 Transformer (可含 MoE) ----
        z_vid, aux_total = self.temporal(f_seq, domain_id=domain_id)   # (B,T,512), aux

        # ---- B: TabM 特征 ----
        z_tabm = self.tabm(x_vec)                        # (B, H_tabm)
        z_tabm = self.tabm_proj(z_tabm)                  # (B, 512)
        if self.moe_tabm:
            z_tabm, aux_t = self.tabm_moe(z_tabm, domain_id=domain_id)  # (B,512)
            aux_total = aux_total + aux_t

        # ---- Cross-Attention 融合 ----
        fused = self.fusion(z_vid, z_tabm)               # (B, 512)

        # ---- 融合 MoE 头（可选） ----
        if self.moe_fused:
            fused, aux_f = self.fused_moe(fused, domain_id=domain_id)  # (B,512)
            aux_total = aux_total + aux_f

        # ---- 分类 ----
        logits = self.head(fused)                        # (B, 4)
        return logits, aux_total

# ====================== 指标与训练循环（兼容 aux_loss） ======================
def f1_per_class(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-9) -> Tuple[np.ndarray, float, float]:
    assert y_true.shape == y_pred.shape
    N, C = y_true.shape
    f1_c = np.zeros(C, dtype=np.float32)
    for c in range(C):
        yt, yp = y_true[:, c], y_pred[:, c]
        tp = np.sum((yt == 1) & (yp == 1))
        fp = np.sum((yt == 0) & (yp == 1))
        fn = np.sum((yt == 1) & (yp == 0))
        prec = tp / (tp + fp + eps)
        rec  = tp / (tp + fn + eps)
        f1_c[c] = 2 * prec * rec / (prec + rec + eps)
    macro_f1 = float(np.mean(f1_c))
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    prec = tp / (tp + fp + 1e-9)
    rec  = tp / (tp + fn + 1e-9)
    micro_f1 = 2 * prec * rec / (prec + rec + 1e-9)
    return f1_c, macro_f1, float(micro_f1)

def average_precision_micro(y_true: np.ndarray, y_prob: np.ndarray, num_thresholds: int = 101) -> float:
    thresholds = np.linspace(0.0, 1.0, num_thresholds)
    precision, recall = [], []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(np.float32)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        precision.append(p); recall.append(r)
    order = np.argsort(recall)
    recall = np.array(recall)[order]
    precision = np.array(precision)[order]
    return float(np.trapz(precision, recall))

LAMBDA_MOE = 0.01  # MoE 辅助损失系数

def train_one_epoch(model, loader, optimizer,
                    pos_weight: Optional[paddle.Tensor] = None,
                    clip_grad_norm: Optional[float] = None):
    model.train()
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits, aux = model(x_vid.astype('float32'), x_vec.astype('float32'))  # ← 接收 aux
        if pos_weight is not None:
            cls = F.binary_cross_entropy_with_logits(logits, y.astype('float32'), pos_weight=pos_weight)
        else:
            cls = F.binary_cross_entropy_with_logits(logits, y.astype('float32'))
        loss = cls + LAMBDA_MOE * aux
        loss.backward()
        if clip_grad_norm is not None:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
        optimizer.step()
        optimizer.clear_grad()
        total_loss += float(loss); total_batches += 1
    return total_loss / max(1, total_batches)

@paddle.no_grad()
def evaluate(model, loader, threshold: float = 0.5):
    model.eval()
    ys, ps = [], []
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits, aux = model(x_vid.astype('float32'), x_vec.astype('float32'))  # ← 接收 aux
        cls = F.binary_cross_entropy_with_logits(logits, y.astype('float32'))
        loss = cls + LAMBDA_MOE * aux
        prob = F.sigmoid(logits).numpy()
        ys.append(y.numpy()); ps.append(prob)
        total_loss += float(loss); total_batches += 1
    y_true = np.concatenate(ys, axis=0)
    y_prob = np.concatenate(ps, axis=0)
    y_pred = (y_prob >= threshold).astype(np.float32)
    per_f1, macro_f1, micro_f1 = f1_per_class(y_true, y_pred)
    ap_micro = average_precision_micro(y_true, y_prob)
    return {
        "loss": total_loss / max(1, total_batches),
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class_f1": per_f1.tolist(),
        "micro_AP": ap_micro
    }

# ====================== 合成数据集（可替换为真实数据） ======================
class ToyTwoModalDataset(Dataset):
    """
    返回：
      x_video: (T=36, C=20, H=20, W=20)
      x_vec:   (424,)
      y:       (4,)  0/1
    """
    def __init__(self, n: int, seed: int = 0):
        super().__init__()
        rng = np.random.default_rng(seed)
        self.n = n
        # 按 (n, T, C, H, W)
        self.video = rng.normal(size=(n, 36, 20, 20, 20)).astype('float32')
        self.vec   = rng.normal(size=(n, 424)).astype('float32')

        # 造标签：对视频先在 H/W 上均值，再在 T 上均值 → (n, C=20)
        vid_hw  = self.video.mean(axis=(3, 4))   # (n, T, C)
        vid_avg = vid_hw.mean(axis=1)            # (n, C)

        # 线性映射到 4 个标签
        Wv = rng.normal(size=(20, 4))            # C→4
        Wt = rng.normal(size=(424, 4))           # 424→4
        logits = vid_avg @ Wv + self.vec @ Wt + rng.normal(scale=0.5, size=(n, 4))
        probs  = 1.0 / (1.0 + np.exp(-logits))
        self.y = (probs > 0.5).astype('float32')

    def __getitem__(self, idx: int):
        x_vid = self.video[idx]  # (T,C,H,W)
        x_vec = self.vec[idx]    # (424,)
        y     = self.y[idx]      # (4,)
        return x_vid, x_vec, y

    def __len__(self):
        return self.n

# ====================== 训练入口（可直接运行） ======================
if __name__ == "__main__":
    paddle.seed(2025)
    # 数据
    train_ds = ToyTwoModalDataset(n=64,  seed=42)
    val_ds   = ToyTwoModalDataset(n=32,  seed=233)
    def collate_fn(batch):
        vids, vecs, ys = zip(*batch)
        return (paddle.to_tensor(np.stack(vids, 0)),   # (B,T,C,H,W)
                paddle.to_tensor(np.stack(vecs, 0)),   # (B,424)
                paddle.to_tensor(np.stack(ys, 0)))     # (B,4)
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=False, collate_fn=collate_fn)
    val_loader   = DataLoader(val_ds,   batch_size=2, shuffle=False, drop_last=False, collate_fn=collate_fn)

    # 类别不平衡权重（可选）
    y_train = np.stack([y for _, _, y in train_ds], 0)
    pos_ratio = np.clip(y_train.mean(axis=0), 1e-3, 1-1e-3)
    pos_weight = paddle.to_tensor(((1-pos_ratio)/pos_ratio).astype('float32'))  # (4,)

    # === 构建模型：三处 MoE 开关（默认只开时序 MoE） ===
    model = TwoModalMultiLabelModel(
        vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
        vec_dim=424,
        d_model=512, nhead=2, n_trans_layers=2, trans_ff=1024,
        tabm_hidden=512, dropout=0.1,
        num_labels=4,
        moe_temporal=True,     # 开：时序 Transformer 的 FFN 位置
        moe_fused=False,       # 关：融合 token MoE 头
        moe_tabm=False         # 关：TabM 投影后 MoE
    )
    optimizer = paddle.optimizer.Adam(learning_rate=3e-4, parameters=model.parameters())

    # 训练（演示用：小 epoch）
    best_macro_f1, best = -1.0, None
    for ep in range(1, 3+1):
        train_loss = train_one_epoch(model, train_loader, optimizer,
                                     pos_weight=pos_weight, clip_grad_norm=1.0)
        val_metrics = evaluate(model, val_loader, threshold=0.5)
        print(f"[Epoch {ep:02d}] train_loss={train_loss:.4f} | "
              f"val_loss={val_metrics['loss']:.4f} | "
              f"macro_f1={val_metrics['macro_f1']:.4f} | "
              f"micro_f1={val_metrics['micro_f1']:.4f} | "
              f"per_class_f1={val_metrics['per_class_f1']} | "
              f"micro_AP={val_metrics['micro_AP']:.4f}")
        if val_metrics["macro_f1"] > best_macro_f1:
            best_macro_f1 = val_metrics["macro_f1"]
            best = {k: v.clone() for k, v in model.state_dict().items()}

    if best is not None:
        model.set_state_dict(best)
        print(f"Loaded best state with macro_f1={best_macro_f1:.4f}")


/tmp/ipython-input-3896222078.py:427: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(precision, recall))


[Epoch 01] train_loss=1.0045 | val_loss=0.8427 | macro_f1=0.1364 | micro_f1=0.2609 | per_class_f1=[0.0, 0.0, 0.0, 0.5454545617103577] | micro_AP=0.4593
[Epoch 02] train_loss=0.6558 | val_loss=1.0130 | macro_f1=0.4563 | micro_f1=0.5342 | per_class_f1=[0.5945945978164673, 0.0, 0.6938775777816772, 0.5365853905677795] | micro_AP=0.5376
[Epoch 03] train_loss=0.2265 | val_loss=1.3452 | macro_f1=0.4934 | micro_f1=0.5625 | per_class_f1=[0.4000000059604645, 0.7234042286872864, 0.6000000238418579, 0.25] | micro_AP=0.5076
Loaded best state with macro_f1=0.4934


In [17]:
# -*- coding: utf-8 -*-
import math
from typing import Optional, Tuple
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader
from paddle.vision.models import resnet18

# ====================== 工具：正弦位置编码 ======================
class SinusoidalPositionalEncoding(nn.Layer):
    def __init__(self, d_model: int, max_len: int = 2048):
        super().__init__()
        pe = np.zeros((max_len, d_model), dtype="float32")
        position = np.arange(0, max_len, dtype="float32")[:, None]
        div_term = np.exp(np.arange(0, d_model, 2, dtype="float32") * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        self.register_buffer("pe", paddle.to_tensor(pe), persistable=False)

    def forward(self, x):  # x: (B, T, D)
        T = x.shape[1]
        return x + self.pe[:T, :]

# ====================== 简化版 TabM（占位，可换成你的实现） ======================
class TabMFeatureExtractor(nn.Layer):
    """占位实现：MLP → (B, H)。可直接替换为你修好的 TabM。"""
    def __init__(self, num_features: int, d_hidden: int = 512, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, d_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_hidden, d_hidden),
            nn.ReLU(),
        )
        self.d_hidden = d_hidden

    def forward(self, x_num: paddle.Tensor):  # (B, 424)
        return self.net(x_num)                # (B, H)

# ====================== ResNet18 特征抽取（逐帧） ======================
class ResNet18FrameEncoder(nn.Layer):
    """将 ResNet18 改为 20 通道输入；输出每帧 512 维特征。"""
    def __init__(self, in_channels: int = 20):
        super().__init__()
        self.backbone = resnet18(pretrained=False)
        # 改首层卷积为 20 通道
        self.backbone.conv1 = nn.Conv2D(in_channels, 64, kernel_size=7, stride=2, padding=3, bias_attr=False)
        # 去掉分类头 fc，保留到 avgpool
        self.avgpool = self.backbone.avgpool  # AdaptiveAvgPool2D(1)
        self.out_dim = 512

    def forward(self, x):  # x: (B*T, C=20, H=20, W=20)
        m = self.backbone
        x = m.conv1(x); x = m.bn1(x); x = F.relu(x); x = m.maxpool(x)
        x = m.layer1(x); x = m.layer2(x); x = m.layer3(x); x = m.layer4(x)
        x = self.avgpool(x)          # (B*T, 512, 1, 1)
        x = paddle.flatten(x, 1)     # (B*T, 512)
        return x

# ====================== MoE 基础实现（Top-k，可开关；使用 gather_nd 修复） ======================
class ExpertFFN(nn.Layer):
    def __init__(self, d_model, d_ff, dropout=0.1, act='relu'):
        super().__init__()
        Act = getattr(F, act) if isinstance(act, str) else act
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.drop = nn.Dropout(dropout)
        self.act = Act
    def forward(self, x):
        return self.fc2(self.drop(self.act(self.fc1(x))))

class MoEConfig:
    def __init__(self,
                 n_experts=8,
                 top_k=1,
                 d_ff=2048,
                 dropout=0.1,
                 router_temp=0.5,
                 balance_loss_w=0.005,
                 entropy_reg_w=-0.005,  # 负值→更尖锐
                 diversity_w=1e-3,
                 sticky_w=0.0,
                 sup_router_w=0.0,
                 use_gumbel=True):
        self.n_experts = n_experts
        self.top_k = top_k
        self.d_ff = d_ff
        self.dropout = dropout
        self.router_temp = router_temp
        self.balance_loss_w = balance_loss_w
        self.entropy_reg_w = entropy_reg_w
        self.diversity_w = diversity_w
        self.sticky_w = sticky_w
        self.sup_router_w = sup_router_w
        self.use_gumbel = use_gumbel

class MoE(nn.Layer):
    """forward(x, domain_id=None) → (y, aux_loss)，支持 (B,T,D) 或 (N,D)"""
    def __init__(self, d_model: int, cfg: MoEConfig):
        super().__init__()
        self.cfg = cfg
        self.router = nn.Linear(d_model, cfg.n_experts)
        self.experts = nn.LayerList([ExpertFFN(d_model, cfg.d_ff, cfg.dropout) for _ in range(cfg.n_experts)])
        self.ln = nn.LayerNorm(d_model)
        self.drop = nn.Dropout(cfg.dropout)

    def _router_probs(self, logits):
        if self.cfg.use_gumbel and self.training:
            u = paddle.uniform(logits.shape, min=1e-6, max=1-1e-6, dtype=logits.dtype)
            g = -paddle.log(-paddle.log(u))
            logits = logits + g
        return F.softmax(logits / self.cfg.router_temp, axis=-1)

    def forward(self, x, domain_id=None):
        orig_shape = x.shape
        if len(orig_shape) == 3:
            B, T, D = orig_shape
            X = x.reshape([B*T, D])
        else:
            X = x
        N, D = X.shape

        logits = self.router(X)             # (N,E)
        probs = self._router_probs(logits)  # (N,E)
        topk_val, topk_idx = paddle.topk(probs, k=self.cfg.top_k, axis=-1)  # (N,k)

        # 专家并行输出
        all_out = paddle.stack([e(X) for e in self.experts], axis=1)         # (N,E,D)

        # === 使用 gather_nd 逐样本选择 top-k 专家 ===
        arangeN = paddle.arange(N, dtype='int64')
        picked_list = []
        for i in range(self.cfg.top_k):
            idx_i = topk_idx[:, i].astype('int64')                   # (N,)
            idx_nd = paddle.stack([arangeN, idx_i], axis=1)          # (N,2) [sample, expert]
            picked_i = paddle.gather_nd(all_out, idx_nd)             # (N,D)
            picked_list.append(picked_i)
        picked = paddle.stack(picked_list, axis=1)                   # (N,k,D)

        # 归一化权重并加权
        w = topk_val / (paddle.sum(topk_val, axis=-1, keepdim=True) + 1e-9)  # (N,k)
        Y = paddle.sum(picked * w.unsqueeze(-1), axis=1)                      # (N,D)

        Y = self.drop(Y)
        Y = self.ln(Y + X)

        # aux loss
        aux = 0.0
        if self.cfg.balance_loss_w > 0:
            mean_prob = probs.mean(axis=0)
            target = paddle.full_like(mean_prob, 1.0 / self.cfg.n_experts)
            aux = aux + self.cfg.balance_loss_w * F.mse_loss(mean_prob, target)
        if self.cfg.entropy_reg_w != 0.0:
            ent = -paddle.sum(probs * (paddle.log(probs + 1e-9)), axis=1).mean()
            aux = aux + self.cfg.entropy_reg_w * ent
        if (domain_id is not None) and (self.cfg.sup_router_w > 0):
            dom = domain_id.reshape([-1])[:N] % self.cfg.n_experts
            aux = aux + self.cfg.sup_router_w * F.cross_entropy(logits, dom)
        if self.cfg.diversity_w > 0 and self.cfg.n_experts > 1:
            # 用 top-1 硬选择近似每个专家接收的样本
            chosen = F.one_hot(topk_idx[:, 0], num_classes=self.cfg.n_experts).astype('float32')  # (N,E)
            denom = chosen.sum(axis=0).clip(min=1.0).unsqueeze(-1)
            means = (all_out * chosen.unsqueeze(-1)).sum(axis=0) / denom                           # (E,D)
            sims = []
            for i in range(self.cfg.n_experts):
                for j in range(i+1, self.cfg.n_experts):
                    si = F.normalize(means[i:i+1], axis=-1)
                    sj = F.normalize(means[j:j+1], axis=-1)
                    sims.append((si*sj).sum())
            if sims:
                aux = aux + self.cfg.diversity_w * paddle.stack(sims).mean()

        if len(orig_shape) == 3:
            Y = Y.reshape([B, T, D])
        return Y, aux

class MoEHead(nn.Layer):
    """单 token MoE 头，用于 fused/tabm 投影后的 (B, D)"""
    def __init__(self, d_model=512, cfg: MoEConfig = None):
        super().__init__()
        self.moe = MoE(d_model, cfg or MoEConfig())
    def forward(self, tok, domain_id=None):
        y, aux = self.moe(tok.unsqueeze(1), domain_id=domain_id)  # (B,1,D)
        return y.squeeze(1), aux

# ====================== 自定义 Transformer Encoder（FFN 可替换为 MoE） ======================
class TransformerEncoderLayerMoE(nn.Layer):
    def __init__(self, d_model=512, nhead=8, d_ff=1024, dropout=0.1,
                 use_moe: bool = True, moe_cfg: MoEConfig = None):
        super().__init__()
        self.use_moe = use_moe
        self.self_attn = nn.MultiHeadAttention(embed_dim=d_model, num_heads=nhead, dropout=dropout)
        self.ln1 = nn.LayerNorm(d_model)
        self.do1 = nn.Dropout(dropout)
        if use_moe:
            self.moe = MoE(d_model, moe_cfg or MoEConfig(d_ff=d_ff, dropout=dropout))
        else:
            self.ffn = nn.Sequential(
                nn.LayerNorm(d_model),
                nn.Linear(d_model, d_ff),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(d_ff, d_model),
            )
            self.do2 = nn.Dropout(dropout)

    def forward(self, x, domain_id=None):  # x: (B,T,D)
        # Self-Attention (pre-norm) —— Paddle MHA 期望 (T,B,D)
        h = self.ln1(x)
        h = paddle.transpose(h, [1, 0, 2])          # (T,B,D)
        sa = self.self_attn(h, h, h)                # (T,B,D)
        sa = paddle.transpose(sa, [1, 0, 2])        # (B,T,D)
        x = x + self.do1(sa)
        aux = 0.0
        if self.use_moe:
            x, aux = self.moe(x, domain_id=domain_id)  # 残差+LN 在 MoE 内部
        else:
            x = x + self.do2(self.ffn(x))              # 残差在这里
        return x, aux

class TemporalTransformerFlexible(nn.Layer):
    def __init__(self, d_model=512, nhead=8, num_layers=2, d_ff=1024, dropout=0.1,
                 max_len=1024, use_moe: bool = True, moe_cfg: MoEConfig = None):
        super().__init__()
        self.pos = SinusoidalPositionalEncoding(d_model, max_len=max_len)
        self.layers = nn.LayerList([
            TransformerEncoderLayerMoE(d_model, nhead, d_ff, dropout,
                                       use_moe=use_moe, moe_cfg=moe_cfg)
            for _ in range(num_layers)
        ])
    def forward(self, x, domain_id=None):  # x: (B,T,D)
        x = self.pos(x)
        aux_total = 0.0
        for layer in self.layers:
            x, aux = layer(x, domain_id=domain_id)
            aux_total = aux_total + aux
        return x, aux_total

# ====================== 多头注意力（支持 q from A, kv from B） ======================
class MultiHeadCrossAttention(nn.Layer):
    def __init__(self, d_model: int, nhead: int = 8, dropout: float = 0.1):
        super().__init__()
        assert d_model % nhead == 0
        self.d_model = d_model
        self.nhead = nhead
        self.d_head = d_model // nhead
        self.Wq = nn.Linear(d_model, d_model)
        self.Wk = nn.Linear(d_model, d_model)
        self.Wv = nn.Linear(d_model, d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(d_model)

    def forward(self, q, kv):
        B, Nq, D = q.shape
        Nk = kv.shape[1]
        q_lin = self.Wq(q); k_lin = self.Wk(kv); v_lin = self.Wv(kv)
        def split_heads(t):
            return t.reshape([B, -1, self.nhead, self.d_head]).transpose([0, 2, 1, 3])
        qh = split_heads(q_lin); kh = split_heads(k_lin); vh = split_heads(v_lin)
        scores = paddle.matmul(qh, kh, transpose_y=True) / math.sqrt(self.d_head)
        attn = F.softmax(scores, axis=-1)
        ctx = paddle.matmul(attn, vh)
        ctx = ctx.transpose([0, 2, 1, 3]).reshape([B, Nq, D])
        out = self.proj(ctx)
        out = self.drop(out)
        return self.ln(out + q)

# ====================== 融合头（双向 Cross-Attn） ======================
class BiModalCrossFusion(nn.Layer):
    """
    输入：
      video_seq: (B, T, D) —— Transformer 后的视频序列
      tabm_tok:  (B, D)    —— TabM token
    """
    def __init__(self, d_model=512, nhead=8, dropout=0.1, fuse_hidden=512):
        super().__init__()
        self.ca_v_from_t = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.ca_t_from_v = MultiHeadCrossAttention(d_model, nhead, dropout)
        self.fuse = nn.Sequential(
            nn.Linear(2 * d_model, fuse_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
        )
        self.out_dim = fuse_hidden

    def forward(self, video_seq, tabm_tok):
        B, T, D = video_seq.shape
        # 池化视频时间维得到 token
        v_tok = video_seq.mean(axis=1, keepdim=True)      # (B,1,D)
        t_tok = tabm_tok.unsqueeze(1)                     # (B,1,D)
        v_upd = self.ca_v_from_t(v_tok, t_tok)            # (B,1,D)
        t_upd = self.ca_t_from_v(t_tok, video_seq)        # (B,1,D)
        fused = paddle.concat([v_upd, t_upd], axis=-1)    # (B,1,2D)
        fused = fused.squeeze(1)                          # (B,2D)
        return self.fuse(fused)                           # (B, F)

# ====================== 总模型（带三个 MoE 开关） ======================
class TwoModalMultiLabelModel(nn.Layer):
    def __init__(self,
                 # 视频模态
                 vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
                 # 结构化模态
                 vec_dim=424,
                 # 维度与结构
                 d_model=512, nhead=2, n_trans_layers=2, trans_ff=1024,
                 tabm_hidden=512, dropout=0.1, num_labels=4,
                 # ===== MoE 开关 =====
                 moe_temporal: bool = True,      # 时序 Transformer 的 FFN 位置
                 moe_fused: bool = False,        # 融合 token 上的小型 MoE 头
                 moe_tabm: bool = False,         # TabM 投影后
                 # ===== MoE 超参（可传入自定义） =====
                 moe_cfg_temporal: MoEConfig = None,
                 moe_cfg_fused: MoEConfig = None,
                 moe_cfg_tabm: MoEConfig = None):
        super().__init__()
        # A: 逐帧 ResNet18
        self.frame_encoder = ResNet18FrameEncoder(in_channels=vid_channels)
        # A: 时序 Transformer（可开/关 MoE）
        self.temporal = TemporalTransformerFlexible(
            d_model=d_model, nhead=nhead, num_layers=n_trans_layers,
            d_ff=trans_ff, dropout=dropout, max_len=vid_frames,
            use_moe=moe_temporal,
            moe_cfg=moe_cfg_temporal or MoEConfig(
                n_experts=8, top_k=1, d_ff=max(trans_ff, 2048), router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            )
        )
        # B: TabM（或你的 TabM）
        self.tabm = TabMFeatureExtractor(vec_dim, d_hidden=tabm_hidden, dropout=dropout)
        self.tabm_proj = nn.Linear(tabm_hidden, d_model)

        # 可选：TabM 分支 MoE 头
        self.moe_tabm = moe_tabm
        if moe_tabm:
            self.tabm_moe = MoEHead(d_model=d_model, cfg=moe_cfg_tabm or MoEConfig(
                n_experts=6, top_k=1, d_ff=1024, router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            ))

        # 融合：双向 Cross-Attention
        self.fusion = BiModalCrossFusion(d_model=d_model, nhead=nhead, dropout=dropout, fuse_hidden=d_model)

        # 可选：融合 token MoE 头
        self.moe_fused = moe_fused
        if moe_fused:
            self.fused_moe = MoEHead(d_model=d_model, cfg=moe_cfg_fused or MoEConfig(
                n_experts=6, top_k=1, d_ff=1024, router_temp=0.5,
                balance_loss_w=0.005, entropy_reg_w=-0.005, diversity_w=1e-3
            ))

        # 分类头
        self.head = nn.Linear(self.fusion.out_dim, num_labels)

    # --- 新增：编码函数，导出融合前的 512 维特征（用于检索库） ---
    def encode(self, x_video, x_vec, domain_id=None):
        """返回融合后的 512 维 token（分类头之前的表示），不经过最终 Linear。"""
        B, T, C, H, W = x_video.shape
        xvt = x_video.reshape([B * T, C, H, W])
        f_frame = self.frame_encoder(xvt)                 # (B*T, 512)
        f_seq = f_frame.reshape([B, T, -1])               # (B, T, 512)
        z_vid, _ = self.temporal(f_seq, domain_id=domain_id)  # (B,T,512)
        z_tabm = self.tabm(x_vec)
        z_tabm = self.tabm_proj(z_tabm)                   # (B,512)
        if self.moe_tabm:
            z_tabm, _ = self.tabm_moe(z_tabm, domain_id=domain_id)
        fused = self.fusion(z_vid, z_tabm)                # (B,512)
        if self.moe_fused:
            fused, _ = self.fused_moe(fused, domain_id=domain_id)
        return fused                                      # (B,512)

    def forward(self, x_video, x_vec, domain_id=None):
        fused = self.encode(x_video, x_vec, domain_id=domain_id)  # (B,512)
        logits = self.head(fused)                                 # (B,4)
        # 为了兼容旧接口，这里返回的 aux 为 0（MoE 的 aux 已在 temporal/tabm_moe/fused_moe 内部求和并丢弃）
        # 如果你想把 MoE 的 aux 在训练里也加入，可把 encode 拆回 forward 的各步并返回累积 aux。
        aux_placeholder = paddle.to_tensor(0.0, dtype='float32')
        return logits, aux_placeholder

# ====================== 指标与训练循环（兼容 aux_loss） ======================
def f1_per_class(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-9) -> Tuple[np.ndarray, float, float]:
    assert y_true.shape == y_pred.shape
    N, C = y_true.shape
    f1_c = np.zeros(C, dtype=np.float32)
    for c in range(C):
        yt, yp = y_true[:, c], y_pred[:, c]
        tp = np.sum((yt == 1) & (yp == 1))
        fp = np.sum((yt == 0) & (yp == 1))
        fn = np.sum((yt == 1) & (yp == 0))
        prec = tp / (tp + fp + eps)
        rec  = tp / (tp + fn + eps)
        f1_c[c] = 2 * prec * rec / (prec + rec + eps)
    macro_f1 = float(np.mean(f1_c))
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    prec = tp / (tp + fp + 1e-9)
    rec  = tp / (tp + fn + 1e-9)
    micro_f1 = 2 * prec * rec / (prec + rec + 1e-9)
    return f1_c, macro_f1, float(micro_f1)

def average_precision_micro(y_true: np.ndarray, y_prob: np.ndarray, num_thresholds: int = 101) -> float:
    thresholds = np.linspace(0.0, 1.0, num_thresholds)
    precision, recall = [], []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(np.float32)
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        precision.append(p); recall.append(r)
    order = np.argsort(recall)
    recall = np.array(recall)[order]
    precision = np.array(precision)[order]
    return float(np.trapz(precision, recall))

LAMBDA_MOE = 0.0  # 这里的 forward 返回 aux=0（如需把 MoE aux 算进去，可按上一版做法）

def train_one_epoch(model, loader, optimizer,
                    pos_weight: Optional[paddle.Tensor] = None,
                    clip_grad_norm: Optional[float] = None):
    model.train()
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits, _ = model(x_vid.astype('float32'), x_vec.astype('float32'))
        if pos_weight is not None:
            cls = F.binary_cross_entropy_with_logits(logits, y.astype('float32'), pos_weight=pos_weight)
        else:
            cls = F.binary_cross_entropy_with_logits(logits, y.astype('float32'))
        loss = cls  # + LAMBDA_MOE * aux  # 此版本不叠加 MoE aux
        loss.backward()
        if clip_grad_norm is not None:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
        optimizer.step()
        optimizer.clear_grad()
        total_loss += float(loss); total_batches += 1
    return total_loss / max(1, total_batches)

# ====================== 检索增强：构建训练库 & 测试时融合 ======================
class Retriever:
    """
    检索库：
      - keys: 训练集的特征 (N,D) —— 取模型 encode() 的融合特征
      - labels: 训练集的标签 (N,C)
    推理：
      - 给定测试特征 (B,D)，计算与 keys 的相似度，取 top-k
      - 得到邻居标签的加权均值 p_knn，按 alpha 融合到模型概率
    """
    def __init__(self, sim_metric: str = 'cos', k: int = 8, alpha: float = 0.3, tau: float = 1.0):
        """
        sim_metric: 'cos' or 'l2'
        k: 近邻数
        alpha: 融合系数，p_final = (1-alpha)*p_model + alpha*p_knn
        tau: 温度（用于 l2 的 softmax(-d/tau) 或 cos 的 softmax(sim/tau)）
        """
        assert sim_metric in ['cos', 'l2']
        self.sim_metric = sim_metric
        self.k = k
        self.alpha = alpha
        self.tau = tau
        self.keys = None     # (N,D)
        self.labels = None   # (N,C)

    @paddle.no_grad()
    def build(self, model: nn.Layer, loader: DataLoader):
        model.eval()
        feats, labs = [], []
        for x_vid, x_vec, y in loader:
            f = model.encode(x_vid.astype('float32'), x_vec.astype('float32'))  # (B,512)
            feats.append(f.numpy())
            labs.append(y.numpy())
        self.keys = paddle.to_tensor(np.concatenate(feats, axis=0)).astype('float32')   # (N,D)
        self.labels = paddle.to_tensor(np.concatenate(labs, axis=0)).astype('float32')  # (N,C)
        # 预归一化（cos 相似度更快；l2 也可复用）
        self.keys_norm = F.normalize(self.keys, axis=-1)

    @paddle.no_grad()
    def query_and_fuse(self, model_probs: paddle.Tensor, test_feat: paddle.Tensor) -> paddle.Tensor:
        """
        model_probs: (B,C)   —— 模型自身概率（sigmoid后的）
        test_feat:   (B,D)   —— 模型 encode 导出的融合特征
        return:      (B,C)   —— 融合后的概率
        """
        assert self.keys is not None, "Call build() before query."
        B, D = test_feat.shape
        # 相似度
        if self.sim_metric == 'cos':
            q = F.normalize(test_feat, axis=-1)                # (B,D)
            sim = paddle.matmul(q, self.keys_norm, transpose_y=True)  # (B,N)
            w = F.softmax(sim / self.tau, axis=-1)             # (B,N)
        else:  # 'l2'
            # ||q-k||^2 = q^2 + k^2 - 2 q·k
            q2 = paddle.sum(test_feat * test_feat, axis=-1, keepdim=True)          # (B,1)
            k2 = paddle.sum(self.keys * self.keys, axis=-1, keepdim=True).transpose([1,0])  # (1,N)
            dot = paddle.matmul(test_feat, self.keys, transpose_y=True)            # (B,N)
            dist2 = q2 + k2 - 2.0 * dot                                            # (B,N)
            w = F.softmax(-dist2 / self.tau, axis=-1)                              # (B,N)

        # 取 top-k（可选：先 topk 再归一化，避免长尾干扰）
        topk_val, topk_idx = paddle.topk(w, k=min(self.k, w.shape[1]), axis=-1)    # (B,k)
        # gather labels
        N, C = self.labels.shape
        b_idx = paddle.arange(B, dtype='int64').unsqueeze(-1).tile([1, topk_val.shape[1]])  # (B,k)
        # 先 gather 权重对应的 labels
        picked_labels = paddle.gather(self.labels, topk_idx.reshape([-1]), axis=0)          # (B*k, C)
        picked_labels = picked_labels.reshape([B, -1, C])                                   # (B,k,C)
        w_norm = topk_val / (paddle.sum(topk_val, axis=-1, keepdim=True) + 1e-9)            # (B,k)
        p_knn = paddle.sum(picked_labels * w_norm.unsqueeze(-1), axis=1)                    # (B,C)

        # 概率融合
        p_final = (1.0 - self.alpha) * model_probs + self.alpha * p_knn
        return p_final.clip(1e-6, 1-1e-6)

@paddle.no_grad()
def evaluate(model, loader, threshold: float = 0.5,
             retriever: Optional[Retriever] = None):
    """
    若 retriever 不为 None：在测试时做 kNN 检索并与模型概率融合。
    """
    model.eval()
    ys, ps = [], []
    total_loss, total_batches = 0.0, 0
    for x_vid, x_vec, y in loader:
        logits, _ = model(x_vid.astype('float32'), x_vec.astype('float32'))
        prob = F.sigmoid(logits)  # (B,C)

        if retriever is not None:
            feat = model.encode(x_vid.astype('float32'), x_vec.astype('float32'))     # (B,D)
            prob = retriever.query_and_fuse(prob, feat)                                # (B,C)

        loss = F.binary_cross_entropy(prob, y.astype('float32'))  # 用概率计算 eval loss
        ys.append(y.numpy()); ps.append(prob.numpy())
        total_loss += float(loss); total_batches += 1

    y_true = np.concatenate(ys, axis=0)
    y_prob = np.concatenate(ps, axis=0)
    y_pred = (y_prob >= threshold).astype(np.float32)
    per_f1, macro_f1, micro_f1 = f1_per_class(y_true, y_pred)
    ap_micro = average_precision_micro(y_true, y_prob)
    return {
        "loss": total_loss / max(1, total_batches),
        "macro_f1": macro_f1,
        "micro_f1": micro_f1,
        "per_class_f1": per_f1.tolist(),
        "micro_AP": ap_micro
    }

# ====================== 合成数据集（可替换为真实数据） ======================
class ToyTwoModalDataset(Dataset):
    """
    返回：
      x_video: (T=36, C=20, H=20, W=20)
      x_vec:   (424,)
      y:       (4,)  0/1
    """
    def __init__(self, n: int, seed: int = 0):
        super().__init__()
        rng = np.random.default_rng(seed)
        self.n = n
        self.video = rng.normal(size=(n, 36, 20, 20, 20)).astype('float32')
        self.vec   = rng.normal(size=(n, 424)).astype('float32')

        # 造标签：对视频先在 H/W 上均值，再在 T 上均值 → (n, C=20)
        vid_hw  = self.video.mean(axis=(3, 4))   # (n, T, C)
        vid_avg = vid_hw.mean(axis=1)            # (n, C)
        Wv = rng.normal(size=(20, 4))            # C→4
        Wt = rng.normal(size=(424, 4))           # 424→4
        logits = vid_avg @ Wv + self.vec @ Wt + rng.normal(scale=0.5, size=(n, 4))
        probs  = 1.0 / (1.0 + np.exp(-logits))
        self.y = (probs > 0.5).astype('float32')

    def __getitem__(self, idx: int):
        return self.video[idx], self.vec[idx], self.y[idx]

    def __len__(self):
        return self.n

# ====================== 训练入口（可直接运行） ======================
if __name__ == "__main__":
    paddle.seed(2025)
    # 数据
    train_ds = ToyTwoModalDataset(n=128, seed=42)
    val_ds   = ToyTwoModalDataset(n=32,  seed=233)

    def collate_fn(batch):
        vids, vecs, ys = zip(*batch)
        return (paddle.to_tensor(np.stack(vids, 0)),   # (B,T,C,H,W)
                paddle.to_tensor(np.stack(vecs, 0)),   # (B,424)
                paddle.to_tensor(np.stack(ys, 0)))     # (B,4)

    train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, drop_last=False, collate_fn=collate_fn)
    val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, drop_last=False, collate_fn=collate_fn)

    # 类别不平衡权重（可选）
    y_train = np.stack([y for _, _, y in train_ds], 0)
    pos_ratio = np.clip(y_train.mean(axis=0), 1e-3, 1-1e-3)
    pos_weight = paddle.to_tensor(((1-pos_ratio)/pos_ratio).astype('float32'))  # (4,)

    # === 模型（MoE 开关按需） ===
    model = TwoModalMultiLabelModel(
        vid_channels=20, vid_h=20, vid_w=20, vid_frames=36,
        vec_dim=424,
        d_model=512, nhead=2, n_trans_layers=2, trans_ff=1024,
        tabm_hidden=512, dropout=0.1,
        num_labels=4,
        moe_temporal=True,     # FFN 位置 MoE
        moe_fused=False,       # 融合处 MoE
        moe_tabm=False         # TabM 处 MoE
    )
    optimizer = paddle.optimizer.Adam(learning_rate=3e-4, parameters=model.parameters())

    # 训练（演示用）
    best_macro_f1, best = -1.0, None
    for ep in range(1, 3+1):
        train_loss = train_one_epoch(model, train_loader, optimizer,
                                     pos_weight=pos_weight, clip_grad_norm=1.0)
        val_metrics = evaluate(model, val_loader, threshold=0.5, retriever=None)
        print(f"[Epoch {ep:02d}] train_loss={train_loss:.4f} | "
              f"val_loss={val_metrics['loss']:.4f} | "
              f"macro_f1={val_metrics['macro_f1']:.4f} | "
              f"micro_f1={val_metrics['micro_f1']:.4f} | "
              f"per_class_f1={val_metrics['per_class_f1']} | "
              f"micro_AP={val_metrics['micro_AP']:.4f}")
        if val_metrics["macro_f1"] > best_macro_f1:
            best_macro_f1 = val_metrics["macro_f1"]
            best = {k: v.clone() for k, v in model.state_dict().items()}

    if best is not None:
        model.set_state_dict(best)
        print(f"Loaded best state with macro_f1={best_macro_f1:.4f}")

    # === 构建检索库（使用训练集） ===
    retr = Retriever(sim_metric='cos', k=8, alpha=0.3, tau=0.5)  # 可改 'l2'
    retr.build(model, DataLoader(train_ds, batch_size=8, shuffle=False, collate_fn=collate_fn))

    # === 测试时启用检索增强 ===
    val_metrics_knn = evaluate(model, val_loader, threshold=0.5, retriever=retr)
    print(f"[RkNN] val_loss={val_metrics_knn['loss']:.4f} | "
          f"macro_f1={val_metrics_knn['macro_f1']:.4f} | "
          f"micro_f1={val_metrics_knn['micro_f1']:.4f} | "
          f"per_class_f1={val_metrics_knn['per_class_f1']} | "
          f"micro_AP={val_metrics_knn['micro_AP']:.4f}")


/usr/local/lib/python3.12/dist-packages/paddle/nn/layer/norm.py:818: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(
/tmp/ipython-input-2448755935.py:419: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return float(np.trapz(precision, recall))


[Epoch 01] train_loss=0.8949 | val_loss=0.7680 | macro_f1=0.4306 | micro_f1=0.5255 | per_class_f1=[0.5454545617103577, 0.5405405163764954, 0.6363636255264282, 0.0] | micro_AP=0.4448
[Epoch 02] train_loss=0.5957 | val_loss=0.9367 | macro_f1=0.4620 | micro_f1=0.5414 | per_class_f1=[0.4444444477558136, 0.7450980544090271, 0.5333333611488342, 0.125] | micro_AP=0.5133
[Epoch 03] train_loss=0.3141 | val_loss=1.1780 | macro_f1=0.3715 | micro_f1=0.4158 | per_class_f1=[0.125, 0.5, 0.6000000238418579, 0.260869562625885] | micro_AP=0.5041
Loaded best state with macro_f1=0.4620
[RkNN] val_loss=0.8189 | macro_f1=0.4900 | micro_f1=0.5390 | per_class_f1=[0.4000000059604645, 0.7599999904632568, 0.5142857432365417, 0.2857142984867096] | micro_AP=0.4978
